# PyHC Core Package Unit Tests

## Helper Functions

In [1]:
import os
import shutil
import xml.etree.ElementTree as ET
import subprocess

def clone_repo(repo_url: str, tag_or_branch: str, target_dir: str):
    """Clone a repository at a specific tag or branch into target_dir."""
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    cmd = ["git", "clone", "--branch", tag_or_branch, repo_url, target_dir]
    subprocess.check_call(cmd)

def run_pytest(test_args: str = "."):
    """
    Run pytest with junitxml output and return True if the command runs.
    test_args can be a directory, file pattern, or additional pytest flags.
    """
    cmd = ["pytest", "--junitxml=test-results.xml"]
    if isinstance(test_args, str):
        cmd.extend(test_args.split())
    else:
        cmd.extend(test_args)
    # Using subprocess.run here to avoid raising exception on test failure
    subprocess.run(cmd, check=False)

def parse_test_results(xml_path: str = "test-results.xml"):
    """Parse a JUnit XML results file and return total, passed, failed, etc."""
    if not os.path.isfile(xml_path):
        # If no test results are produced, treat it as 0 tests run
        return 0, 0, 0, 0, 0

    tree = ET.parse(xml_path)
    root = tree.getroot()

    total = errors = failures = skipped = 0
    if root.tag == 'testsuites':
        for testsuite in root.findall('testsuite'):
            total += int(testsuite.attrib.get('tests', 0))
            errors += int(testsuite.attrib.get('errors', 0))
            failures += int(testsuite.attrib.get('failures', 0))
            skipped += int(testsuite.attrib.get('skipped', 0))
    elif root.tag == 'testsuite':
        total = int(root.attrib.get('tests', 0))
        errors = int(root.attrib.get('errors', 0))
        failures = int(root.attrib.get('failures', 0))
        skipped = int(root.attrib.get('skipped', 0))
    else:
        raise RuntimeError(f'Unexpected root tag in test-results.xml: {root.tag}')

    passed = total - errors - failures
    pass_rate = (passed / total) * 100 if total > 0 else 0
    return total, passed, failures, errors, skipped, pass_rate

def check_pass_rate(pass_rate: float, threshold: float, package_name: str):
    """Check if pass_rate meets the threshold, raise RuntimeError if not."""
    if pass_rate < threshold:
        raise RuntimeError(f'{package_name} tests failed pass rate threshold ({threshold}%).')

def print_test_summary(package_name: str, total: int, passed: int, failures: int, errors: int, skipped: int, pass_rate: float):
    """Print a summary of test results."""
    print(f'{package_name} Tests:')
    print(f'Total tests: {total}')
    print(f'Passed (including skipped): {passed}')
    print(f'Failures: {failures}')
    print(f'Errors: {errors}')
    print(f'Skipped: {skipped}')
    print(f'Pass rate: {pass_rate:.2f}%')

## 1. Test HAPI Client

In [ ]:
import hapiclient

hapiclient_version_tag = hapiclient.__version__.split('.dev')[0]

# Clone repo
clone_repo("https://github.com/hapi-server/client-python.git", f"v{hapiclient_version_tag}", "client-python")

# Run tests
os.chdir("client-python")
run_pytest("hapiclient/test/")
total, passed, failures, errors, skipped, pass_rate = parse_test_results("test-results.xml")
os.chdir("..")

print_test_summary("HAPI Client", total, passed, failures, errors, skipped, pass_rate)
check_pass_rate(pass_rate, 85, "HAPI Client")

## 2. Test PlasmaPy

In [ ]:
import plasmapy

plasmapy_version_tag = plasmapy.__version__.split('.dev')[0]

# Clone repo
clone_repo("https://github.com/PlasmaPy/PlasmaPy.git", f"v{plasmapy_version_tag}", "PlasmaPy")

# Run tests
os.chdir("PlasmaPy")
run_pytest("--continue-on-collection-errors --ignore=tests/utils/data/test_downloader.py")
total, passed, failures, errors, skipped, pass_rate = parse_test_results("test-results.xml")
os.chdir("..")

print_test_summary("PlasmaPy", total, passed, failures, errors, skipped, pass_rate)
check_pass_rate(pass_rate, 98, "PlasmaPy")

## 3. Test pysat

In [ ]:
import pysat
import os

# Ensure a data directory for pysat
if not os.path.exists('pysatData'):
    os.makedirs('pysatData')
pysat.params['data_dirs'] = [os.path.abspath('pysatData')]

# Run tests
run_pytest("--pyargs pysat.tests")
total, passed, failures, errors, skipped, pass_rate = parse_test_results("test-results.xml")

print_test_summary("pysat", total, passed, failures, errors, skipped, pass_rate)
check_pass_rate(pass_rate, 98, "pysat")

## 4. Test PySPEDAS

In [2]:
# Run tests
run_pytest("--pyargs pyspedas")
total, passed, failures, errors, skipped, pass_rate = parse_test_results("test-results.xml")

print_test_summary("pySPEDAS", total, passed, failures, errors, skipped, pass_rate)
check_pass_rate(pass_rate, 90, "pySPEDAS")

============================= test session starts ==============================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0
Matplotlib: 3.10.0
Freetype: 2.6.1
rootdir: /home/jovyan
plugins: anyio-4.8.0, asdf-4.0.0, hypothesis-6.124.7, arraydiff-0.6.1, astropy-0.11.0, astropy-header-0.2.2, cov-6.0.0, datadir-1.5.0, doctestplus-1.4.0, filter-subpackage-0.2.0, mock-3.14.0, mpl-0.17.0, ordering-0.6, regressions-2.7.0, remotedata-0.4.1, rerunfailures-15.0, xdist-3.6.1
collected 64 items

............                       [ 18%]
.FFFF                                     [ 26%]
.....                               [ 37%]
..........ss.............                [ 81%]
mth5/tests/test_mth5_import.py .                                         [ 82%]
projects/elfin/tests/test_epd_calibration.py .                           [ 84%]
....                                 [ 90%]
....                                 [ 96%]
.                                     [ 98%]
utilities/tests/tes

## 5. Test SpacePy

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
import spacepy

spacepy_version_tag = spacepy.__version__.split('.dev')[0]

# Clean up if previous clone exists
if os.path.exists('spacepy'):
    shutil.rmtree('spacepy')

!git clone --branch "release-{spacepy_version_tag}" "https://github.com/spacepy/spacepy.git"
%cd spacepy/tests

# Run selected test files
!python test_ae9ap9.py
!python test_coordinates.py
!python test_ctrans.py
!python test_datamanager.py
!python test_datamodel.py
!python test_empiricals.py
!python test_igrf.py
!python test_irbempy.py
!python test_lanlstar.py
!python test_lib.py
!python test_omni.py
!python test_plot.py
!python test_plot_utils.py
!python test_poppy.py
!python test_pybats.py
!python test_pycdf.py
!python test_pycdf_istp.py
!python test_rst.py
!python test_seapy.py
!python test_spectrogram.py
!python test_testing.py
!python test_time.py
!python test_toolbox.py

# SpacePy doesn't produce a single junit xml via these commands.
# If you need pass/fail info, consider modifying tests or using pytest.
# For now, just assume no major failures if return code is 0.

print("SpacePy tests executed. Note: No automatic pass rate calculation done here.")
print("If you need pass/fail rate, you must run tests via pytest with --junitxml.")
%cd ../..

## 6. Test SunPy

In [ ]:
import sunpy

# Run tests
run_pytest("--pyargs sunpy")
total, passed, failures, errors, skipped, pass_rate = parse_test_results("test-results.xml")

print_test_summary("SunPy", total, passed, failures, errors, skipped, pass_rate)
check_pass_rate(pass_rate, 98, "SunPy")